In [1]:
import tempfile
from typing import Dict, Iterable, List, Tuple

import allennlp
import torch
from allennlp.data import DataLoader, DatasetReader, Instance, Vocabulary
from allennlp.data.fields import LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer
from allennlp.models import Model
from allennlp.modules import TextFieldEmbedder, Seq2VecEncoder
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2vec_encoders import BertPooler
from allennlp.nn import util
from allennlp.training.trainer import GradientDescentTrainer, Trainer
from allennlp.training.optimizers import AdamOptimizer
from allennlp.training.metrics import CategoricalAccuracy

from os.path import join as pathjoin
import pandas as pd
from allennlp.predictors import TextClassifierPredictor
from allennlp.training.metrics import CategoricalAccuracy
import numpy as np

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_bert_base_cased'
transformer_model = 'bert-base-cased'
MAX_TOKENS = 510

In [3]:
#!head -100 '/home/mlepekhin/data/multi_train' > '{DATA_DIR}/multi_train'
#!head -10 '/home/mlepekhin/data/multi_test' > '{DATA_DIR}/multi_test'

In [4]:
#token_indexer = PretrainedTransformerIndexer(model_name=transformer_model)

In [5]:
class ClassificationDatasetReader(DatasetReader):
    def __init__(self,
                 lazy: bool = False,
                 tokenizer: Tokenizer = None,
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_tokens: int = None):
        super().__init__(lazy)
        self.tokenizer = tokenizer or PretrainedTransformerTokenizer(transformer_model, max_length=MAX_TOKENS)
        self.token_indexers = token_indexers or {'bert_tokens': PretrainedTransformerIndexer(transformer_model)}
        self.max_tokens = max_tokens
        
    def text_to_instance(self, string: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(string)
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"text": sentence_field}
        if label is not None:
            fields["label"] = LabelField(label)
        return Instance(fields)

    def _read(self, file_path: str) -> Iterable[Instance]:
        dataset_df = pd.read_csv(file_path)
        for text, label in zip(dataset_df['text'], dataset_df['target']):
            yield self.text_to_instance(text, label)

In [6]:
class SimpleClassifier(Model):
    def __init__(self,
                 vocab: Vocabulary,
                 embedder: TextFieldEmbedder,
                 encoder: Seq2VecEncoder):
        super().__init__(vocab)
        self.embedder = embedder 
        num_labels = vocab.get_vocab_size("labels")
        self.encoder = encoder
        self.classifier = torch.nn.Linear(encoder.get_output_dim(), num_labels)
        self.accuracy = CategoricalAccuracy()
        

    def forward(self,
                text: Dict[str, torch.Tensor],
                label: torch.Tensor=None) -> Dict[str, torch.Tensor]:
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        #print("embed shape", embedded_text.shape)
        # Shape: (batch_size, num_tokens)
        mask = util.get_text_field_mask(text)
        #print("mask shape", mask.shape)
        # Shape: (batch_size, encoding_dim)
        encoded_text = self.encoder(embedded_text, mask)
        # Shape: (batch_size, num_labels)
        logits = self.classifier(encoded_text)
        # Shape: (batch_size, num_labels)
        probs = torch.nn.functional.softmax(logits, dim=-1)
        if label is not None:
            loss = torch.nn.functional.cross_entropy(logits, label)
            self.accuracy(logits, label)
            return {'loss': loss, 'probs': probs}
        else:
            return {'probs': probs}
    
    def get_metrics(self, reset: bool = True) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}


def read_data(reader: DatasetReader, train_path: str, val_path: str) -> Tuple[Iterable[Instance], Iterable[Instance]]:
    print("Reading data")
    training_data = reader.read(train_path)
    validation_data = reader.read(val_path)
    return training_data, validation_data


def build_vocab(instances: Iterable[Instance]) -> Vocabulary:
    print("Building the vocabulary")
    return Vocabulary.from_instances(instances)


def build_model(vocab: Vocabulary) -> Model:
    print("Building the model")
    vocab_size = vocab.get_vocab_size("tokens")
    #embedder = BasicTextFieldEmbedder(
    #    {"tokens": Embedding(embedding_dim=10, num_embeddings=vocab_size)})
    embedding = PretrainedTransformerEmbedder(model_name=transformer_model)
    embedder = BasicTextFieldEmbedder(token_embedders={'bert_tokens': embedding})
    encoder = BertPooler(transformer_model)
    return SimpleClassifier(vocab, embedder, encoder)

def build_dataset_reader() -> DatasetReader:
    return ClassificationDatasetReader()

def run_training_loop():
    dataset_reader = build_dataset_reader()

    # These are a subclass of pytorch Datasets, with some allennlp-specific
    # functionality added.
    train_data, dev_data = read_data(dataset_reader)

    vocab = build_vocab(train_data + dev_data)
    model = build_model(vocab)

    # This is the allennlp-specific functionality in the Dataset object;
    # we need to be able convert strings in the data to integers, and this
    # is how we do it.
    train_data.index_with(vocab)
    dev_data.index_with(vocab)

    # These are again a subclass of pytorch DataLoaders, with an
    # allennlp-specific collate function, that runs our indexing and
    # batching code.
    train_loader, dev_loader = build_data_loaders(train_data, dev_data)

    # You obviously won't want to create a temporary file for your training
    # results, but for execution in binder for this course, we need to do this.
    with tempfile.TemporaryDirectory() as serialization_dir:
        trainer = build_trainer(
            model,
            serialization_dir,
            train_loader,
            dev_loader
        )
        print("Starting training")
        trainer.train()
        print("Finished training")
    return trainer


# The other `build_*` methods are things we've seen before, so they are
# in the setup section above.
def build_data_loaders(
    train_data: torch.utils.data.Dataset,
    dev_data: torch.utils.data.Dataset,
) -> Tuple[allennlp.data.DataLoader, allennlp.data.DataLoader]:
    # Note that DataLoader is imported from allennlp above, *not* torch.
    # We need to get the allennlp-specific collate function, which is
    # what actually does indexing and batching.
    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    dev_loader = DataLoader(dev_data, batch_size=16, shuffle=False)
    return train_loader, dev_loader


def build_trainer(
    model: Model,
    serialization_dir: str,
    train_loader: DataLoader,
    dev_loader: DataLoader,
    num_epochs: int = 1,
    cuda_device: int = -1
) -> Trainer:
    parameters = [
        [n, p]
        for n, p in model.named_parameters() if p.requires_grad
    ]
    optimizer = AdamOptimizer(parameters, lr=0.000025)
    trainer = GradientDescentTrainer(
        model=model,
        serialization_dir=serialization_dir,
        data_loader=train_loader,
        validation_data_loader=dev_loader,
        num_epochs=num_epochs,
        optimizer=optimizer,
        cuda_device=cuda_device,
    )
    return trainer

In [7]:
#trainer = run_training_loop()

dataset_reader = build_dataset_reader()

train_data, dev_data = read_data(
    dataset_reader, 
    pathjoin(DATA_DIR, "en_train"), 
    pathjoin(DATA_DIR, "en_test")
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data



Building the vocabulary


In [8]:
'''
tokenizer = PretrainedTransformerTokenizer(model_name=transformer_model, max_length=MAX_TOKENS)
token_indexer = PretrainedTransformerIndexer(model_name=transformer_model)
#text = "Здравствуй, дорогой друг! Я очень рад тебя видеть. Что нового?"
text = 'Привет! ' * 150
tokens = tokenizer.tokenize(text)
print(len(tokens))
#print("Transformer tokens:", tokens)
#print(type(tokens))

text_field = TextField(tokens, {'bert_tokens': token_indexer})
text_field.index(vocab)
token_tensor = text_field.as_tensor(text_field.get_padding_lengths())
print("Transformer tensors:", token_tensor)

embedding = PretrainedTransformerEmbedder(model_name=transformer_model)

embedder = BasicTextFieldEmbedder(token_embedders={'bert_tokens': embedding})

tensor_dict = text_field.batch_tensors([token_tensor])
print('tensor_dict', tensor_dict)
embedded_tokens = embedder(tensor_dict)
print("Transformer embedded tokens:", embedded_tokens)

embedded_tokens.shape
'''

'\ntokenizer = PretrainedTransformerTokenizer(model_name=transformer_model, max_length=MAX_TOKENS)\ntoken_indexer = PretrainedTransformerIndexer(model_name=transformer_model)\n#text = "Здравствуй, дорогой друг! Я очень рад тебя видеть. Что нового?"\ntext = \'Привет! \' * 150\ntokens = tokenizer.tokenize(text)\nprint(len(tokens))\n#print("Transformer tokens:", tokens)\n#print(type(tokens))\n\ntext_field = TextField(tokens, {\'bert_tokens\': token_indexer})\ntext_field.index(vocab)\ntoken_tensor = text_field.as_tensor(text_field.get_padding_lengths())\nprint("Transformer tensors:", token_tensor)\n\nembedding = PretrainedTransformerEmbedder(model_name=transformer_model)\n\nembedder = BasicTextFieldEmbedder(token_embedders={\'bert_tokens\': embedding})\n\ntensor_dict = text_field.batch_tensors([token_tensor])\nprint(\'tensor_dict\', tensor_dict)\nembedded_tokens = embedder(tensor_dict)\nprint("Transformer embedded tokens:", embedded_tokens)\n\nembedded_tokens.shape\n'

In [9]:
model = build_model(vocab)

Building the model


In [10]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

0


In [11]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.
with tempfile.TemporaryDirectory() as serialization_dir:
    trainer = build_trainer(
        model,
        serialization_dir,
        train_loader,
        dev_loader,
        10,
        cuda_device=cuda_device
    )
    print("Starting training")
    trainer.train()
    print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training



Finished training


In [12]:
import torch

torch.save(model.state_dict, pathjoin(MODELS_DIR, MODEL_ID, 'model'))

/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/serialization.py:401: UserWarning: Couldn't retrieve source code for container of type SimpleClassifier. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "


In [13]:
vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))

In [19]:
id_to_label = []
with open(pathjoin(MODELS_DIR, MODEL_ID, 'vocab', 'labels.txt')) as vocab_in:
    for line in vocab_in:
        id_to_label.append(line.strip())
print(id_to_label)

['A1', 'A12', 'A7', 'A16', 'A8', 'A22', 'A4', 'A11', 'A14', 'A9', 'A17']


In [20]:
def predict_classes(sentence_list):
    predictor = TextClassifierPredictor(model, dataset_reader=build_dataset_reader())
    result = [id_to_label[np.argmax(predictor.predict(sentence)["probs"])]\
              for sentence in sentence_list]
    return result

In [21]:
predict_classes(['Здесь должно быть ваше сообщение',
                 'Коты - лучшие домашние животные. К такому выводу пришли эксперты из издания New York Times',
                 'It is no more than what it is.',
                 'Жила я как-то с парнем. Я только вот на днях уволилась с работы, так как мне тяжело было работать сутки через сутки, должна была выходить на другую работу. И именно в этот период, мне сильно поплохело, начались жуткие головные боли, слабость, обмороки. Парень настоял, что нужно срочно вызывать врача. Приехала скорая, фельдшер мужик лет 50 весь седой. Позадавал вопросы мне, где болит, как болит, и зачем болит? Кто такая вообще по жизни, и чем занимаюсь? Смерил давление, температуру, написал что-то в своих бумагах, и дав лишь рекомендацию: "больше отдыхайте, пейте воду, гуляйте на свежем воздухе" пошёл на выход.'])

['A16', 'A16', 'A1', 'A22']

This is the time to interpret our simple classifier.

In [22]:
from allennlp.interpret.saliency_interpreters import SmoothGradient

In [24]:
predictor = TextClassifierPredictor(model, dataset_reader=build_dataset_reader())
smooth_grad_interpr = SmoothGradient(predictor)

In [25]:
smooth_grad_interpr.saliency_interpret_from_json({'sentence': 'Жила я как-то с парнем. Я только вот на днях уволилась с работы, так как мне тяжело было работать сутки через сутки, должна была выходить на другую работу. И именно в этот период, мне сильно поплохело, начались жуткие головные боли, слабость, обмороки. Парень настоял, что нужно срочно вызывать врача. Приехала скорая, фельдшер мужик лет 50 весь седой. Позадавал вопросы мне, где болит, как болит, и зачем болит? Кто такая вообще по жизни, и чем занимаюсь? Смерил давление, температуру, написал что-то в своих бумагах, и дав лишь рекомендацию: "больше отдыхайте, пейте воду, гуляйте на свежем воздухе" пошёл на выход.'})

{'instance_1': {'grad_input_1': [0.012206169837410004,
   0.0015257712296762504,
   0.003242263863062032,
   0.003051542459352501,
   0.003051542459352501,
   0.004577313689028751,
   0.003814428074190626,
   0.0015257712296762504,
   0.0003814428074190626,
   0.0015257712296762504,
   0.007628856148381252,
   0.001907214037095313,
   0.004958756496447814,
   0.0007628856148381252,
   0.0015257712296762504,
   0.0026700996519334382,
   0.0003814428074190626,
   0.003051542459352501,
   0.003051542459352501,
   0.004577313689028751,
   0.018309254756115004,
   0.007628856148381252,
   0.0026700996519334382,
   0.004195870881609689,
   0.0015257712296762504,
   0.003814428074190626,
   0.0,
   0.0022886568445143756,
   0.0015257712296762504,
   0.0007628856148381252,
   0.001907214037095313,
   0.0015257712296762504,
   0.0,
   0.0011443284222571878,
   0.0015257712296762504,
   0.003051542459352501,
   0.0015257712296762504,
   0.0,
   0.0015257712296762504,
   0.0015257712296762504,
  